# "How to match images taken from really extreme viewpoints?"
> "Until you succeed, try harder."
- toc: false
- image: images/transition.png
- branch: master
- badges: true
- comments: true
- hide: false
- search_exclude: false

### In this post

1. What to do, if you are in a desperate need of matching this particular image pair?
2. What are the limitations of the affine-covariant detectors like Hessian-Affine or HesAffNet?
3. ASIFT: brute-force affine view synthesis
4. Do as little as possible: MODS
5. What is the key factor of affine view synthesis? Ablation study

# How to match images taken from really extreme viewpoints?

Standard wide-baseline stereo or 3d reconstruction pipelines work well in the many situations. Even if some image pair is not matched, it is usually not a problem. For example, one could match images from very different viewpoints, if there is a sequence of images in between, as shown in Figure below, from "From Single Image Query to Detailed 3D Reconstruction" paper\cite{SingleImage3dRec2015}.

![](2020-08-04-affine-view-synthesis_files/att_00000.png "One could handle extreme viewpoint changes by proxy images. Figure from From Single Image Query to Detailed 3D Reconstruction")

However, that might not always be possible. For example, the number of pictures is limited because they historical and there is no way how one could go and take more without inventing a time machine. 


What to do? One way would be to use [affine features](https://ducha-aiki.github.io/wide-baseline-stereo-blog/2020/07/17/affine-correspondences.html) like Hessian-AffNet\cite{AffNet2018} or MSER\cite{MSER2002}. However, they help only up to some extent and what if the view, we need to match are more extreme?

![](2020-08-04-affine-view-synthesis_files/MODS-match-historically.png "Non-matchable by standard methods pair of historical photographies. Matched only with help of affine view synthesis in the MODS framework. Images from Location recognition over large time lags dataset")

The image pair above is from "Location recognition over large time lags dataset" paper \cite{LostInPast2015}.


The solution is to simulate real viewpoint change by affine or perspective warps of the current image. This idea was first proposed by Lepetit and Fua in 2006\cite{AffineTree2006}. You can think about it as a special version of test-time augmentation, popular nowadays in deep learning. Later affine view synthesis for wide baseline stereo was extended and mathematically justified by Morel & Yu in ASIFT paper\cite{ASIFT2009}. They proved that perspective image warps are can be approximated by synthetic affine views.

## What is wrong with affine-covariant local detectors?

One could say that the goal of affine-covariant detectors like MSER, Hessian-Affine or Hessian-AffNet is to detect the same region on a planar surface, regardless the camera angle change.
It is true to some extent, as we demostrate on toy example below with Hessian-Affine feature.

![](2020-08-04-affine-view-synthesis_files/together_single.png "Hessian-Affine detector detects the same support region under synthetic tilt")

The problem arises, when the image content, e.g. 3 blobs on the figure below are situated close to each other, so under the tilt transform the merge into a single blob. So it is not the shape of region, which is detected incorrectly, but the center of the features themselves. For clarity, we omited affine shape estimation on the image below.

![](2020-08-04-affine-view-synthesis_files/blobs_merging.png "When blobs are close to each other, they merge together when picture is taked from a side viewpoint. This results in failure of the blob detector.")

## ASIFT: brute-force affine view synthesis

So, to solve the problem explained above, Morel & Yu \cite{ASIFT2009} proposed to do a lot affine warps of each image, as shown on the Figure below, as match each view against all others, which is $O(n^2)$ complexity, where $n$ is number of views generated. 


![](2020-08-04-affine-view-synthesis_files/att_00006.png "ASIFT algorithm: generate a lot of synthetic views, match all to all. Figure from Fast Affine Invariant Image Matching")

The motivation do doing so it that assuming, original image to be a fronto-parallel one, to cover viewsphere really dense, as shown in the Figure below.

![](2020-08-04-affine-view-synthesis_files/att_00007.png "Viewsphere covering by ASIFT. Figure from Fast Affine Invariant Image Matching")

This leads to impressive performance on a very challenging image pairs, see an example below


![image.png](2020-08-04-affine-view-synthesis_files/att_00009.png "ASIFT find a lot of correspondences on challenging pair. Figure from ASIFT: An Algorithm for Fully Affine Invariant Comparison")

In this section I have used great illustrations done by [Mariano Rodríguez](https://rdguez-mariano.github.io/) for his paper "Fast Affine Invariant Image Matching" \cite{FastASIFT2018}. Please, checkout his [blog](https://rdguez-mariano.github.io/). 

## MODS: do as little as possible 

The main drawback of ASIFT algorithm is a huge computational cost: 82 views are generated regardless of the image pair difficulty. To overcome this, we proposed MODS\cite{MODS2015} algorithm: Matching with On-Demand Synthesis. 

![](2020-08-04-affine-view-synthesis_files/att_00004.png "MODS algorithm: synthetize more views until match")


One starts with the fastest detector-descriptor without view synthesys and then uses more and more computationally expensive methods if needed. Moreover, by using affine-covariant detectors like MSER or Hessian-Affine, one could synthetise significantly less views, saving computations spent on local descriptor and matching. 

This, together with [FGINN matching strategy](https://medium.com/@ducha.aiki/how-to-match-to-learn-or-not-to-learn-part-2-1ab52ede2022), specifically designed for the handling re-detections, MODS is able to match more challenging image pairs in less time than ASIFT.

![](2020-08-04-affine-view-synthesis_files/att_00010.png "MODS outperforms ASIFT on a several dataset both in terms of speed and quality")

## Why does affine synthesis help?

Despite that ASIFT and other view-synthesis based approaches are know more than decade, we are not aware of a study, why does affine synthesis helps in practice. Could one get a similar performance without view synthesis?
Specificallly:

0. May it be that the most of improvements come from the fact that we have much more features? That is why we fix the number of features for all approaches.
1. Some regions from ASIFT, when reprojected to the original image, are quite narrow. Could we get them just by removing edge-like feature filtering, which is done in SIFT, Hessian and other detectors.  Denoted **+edge**
2. Instead of doing affine view synthesis, one could directly use the same affine parameters to get the affine regions to describe, so the each keypoint would have several associated regions+descriptors. Denoted **+MD**
3. Using AffNet to directly estimated local affine shape without multiple descriptors. Denoted **+AffNet**
4. Combine (1), (2) and (3).



So, we did the study on HPatches Sequences dataset, the hardest image pairs (1-6) of viewpoint subset. The metric is similar to one used in the "[Image Matching across Wide Baselines: From Paper to Practice](https://arxiv.org/abs/2003.01587)" and CVPR 2020 [RANSAC in 2020](http://cmp.felk.cvut.cz/cvpr2020-ransac-tutorial/) - mean average accuracy of the estimated homography.

![](2020-08-04-affine-view-synthesis_files/att_00011.png " mean average accuracy of the estimated homography - used metric")

We run Hessian detector with RootSIFT descriptor, FLANN matching and LO-RANSAC, as implemented in [MODS](https://github.com/ducha-aiki/mods-light-zmq). Features are sorted according the the detector response and their total number is clipped to 2048 or 8000 to ensure that the improvements do not come from just having more features.

Note, that we do not study, if view synthesis helps for the regular image pairs - it might actually hurt performance, similarly to [affine features](https://ducha-aiki.github.io/wide-baseline-stereo-blog/2020/07/17/affine-correspondences.html). 
Instead we are focusing on the case, when view synthesis definitely helps: matching obscure views of the mostly planar scenes. 

### 8000 feature budget

Results are in Figure below. Indeed, all of the factors: detecting more edge-like features, having multiple descriptors or better affine shape improve results over the plain Hessian detector, but even all of the combined are not good enough to match performance of the affine view synthesis + plain Hessian detector.

But the best setup is to use both Hessian-AffNet and view synthesis. 

![](2020-08-04-affine-view-synthesis_files/8k_budget.png "Ablation study of the different benefit from the affine view synthesis: ability to detect elongated blobs, multiple affine decriptors or everything together. Feature budget: 8k")

### 2048 feature budget

The picture is a bit different in a small feature budget: neither multiple-(affine)-descriptors per keypoint, nor allowing edge-like feature help. From other hand, affine view synthesis still improves results of the Hessian. And, again, the best performance is achieved with combination of view synthesis and AffNet shape estimation.

![](2020-08-04-affine-view-synthesis_files/2k_budget.png "Ablation study of the different benefit from the affine view synthesis: ability to detect elongated blobs, multiple affine decriptors or everything together. Feature budget: 8k")

## Summary

Affine view synthesis helps for matching challenging image pairs and its improvement are not just because of more local features used. It can be done effective and efficient -- in the iterative MODS framework.


# References

[<a id="cit-SingleImage3dRec2015" href="#call-SingleImage3dRec2015">SingleImage3dRec2015</a>] J.L. Schonberger, F. Radenovic, O. Chum <em>et al.</em>, ``_From Single Image Query to Detailed 3D Reconstruction_'', Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (CVPR), June 2015.

[<a id="cit-AffNet2018" href="#call-AffNet2018">AffNet2018</a>] D. Mishkin, F. Radenovic and J. Matas, ``_Repeatability is Not Enough: Learning Affine Regions via Discriminability_'', ECCV,  2018.

[<a id="cit-MSER2002" href="#call-MSER2002">MSER2002</a>] J. Matas, O. Chum, M. Urban <em>et al.</em>, ``_Robust Wide Baseline Stereo from Maximally Stable Extrema Regions_'', BMVC,  2002.

[<a id="cit-LostInPast2015" href="#call-LostInPast2015">LostInPast2015</a>] Fernando Basura, Tommasi Tatiana and Tuytelaars Tinne, ``_Location recognition over large time lags_'', Computer Vision and Image Understanding, vol. 139, number , pp. ,  2015.

[<a id="cit-AffineTree2006" href="#call-AffineTree2006">AffineTree2006</a>] Lepetit Vincent and Fua Pascal, ``_Keypoint Recognition Using Randomized Trees_'', IEEE Trans. Pattern Anal. Mach. Intell., vol. 28, number 9, pp. , sep 2006.

[<a id="cit-ASIFT2009" href="#call-ASIFT2009">ASIFT2009</a>] Morel Jean-Michel and Yu Guoshen, ``_ASIFT: A New Framework for Fully Affine Invariant Image Comparison_'', SIAM J. Img. Sci., vol. 2, number 2, pp. , apr 2009.

[<a id="cit-FastASIFT2018" href="#call-FastASIFT2018">FastASIFT2018</a>] Rodríguez Mariano, Delon Julie and Morel Jean-Michel, ``_Fast Affine Invariant Image Matching_'', Image Processing On Line, vol. 8, number , pp. ,  2018.

[<a id="cit-MODS2015" href="#call-MODS2015">MODS2015</a>] Mishkin Dmytro, Matas Jiri and Perdoch Michal, ``_MODS: Fast and robust method for two-view matching _'', Computer Vision and Image Understanding , vol. , number , pp. ,  2015.

